In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.espncricinfo.com/records/tournament/batting-most-runs-career/icc-men-s-t20-world-cup-2024-15946"
r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")

# Find the table
box = soup.find("table", class_="ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide")

# Extract header row
thead = box.find("thead", class_="ds-bg-fill-content-alternate ds-text-left")
headers = [td.find("span").text.strip() for td in thead.find_all("td")]

# Extract data rows
rows = box.find_all("tr")[1:]  # Skip the header row

Row = []
for row in rows:
    # Extract each cell (td) within the row
    cells = row.find_all("td")
    # Get text from each cell and strip extra spaces
    row_data = [cell.text.strip() for cell in cells]
    Row.append(row_data)

# Create DataFrame with headers and rows
df = pd.DataFrame(Row, columns=headers)

# Add "Team" column: Assuming the team name is after the player in the "Player" column
# Example: Player1 (TeamName), Player2 (TeamB), etc.

# Extract the team name after the player (assuming it is in parentheses)
df['Team'] = df['Player'].str.extract(r'\((.*?)\)', expand=False)

# Create new column "Player" with player name only (without team)
df['Player'] = df['Player'].str.split('(').str[0].str.strip()

# Reorder the columns so that "Team" comes after "Player"
df = df[['Player', 'Team'] + [col for col in df.columns if col not in ['Player', 'Team']]]

# Print DataFrame
df#.to_csv("wc_bat.csv")
